# Ollama Python

Ollama offers a way to interact with open source large language models
interactively. The easiest way to do that is with the Ollama Chatbot app
in Strudel2, but if you need more power, you can use the Ollama python
library.

In this example we are going to use Ollama to summarise Wikipedia
articles from the Wikitext 2 dataset.

## Preparing the dataset

The Wikitext 2 dataset is easily available through the Hugging Face
library ‘datasets’, but rather than splitting the dataset by article
like we need the data is split by paragraph. This means we’ll need to
reprocess the dataset back into articles for our example.

In [38]:
from datasets import load_dataset
ds = load_dataset("Salesforce/wikitext", "wikitext-2-raw-v1")
ds["test"]["text"][:10]

['',
 ' = Robert Boulter = \n',
 '',
 ' Robert Boulter is an English film , television and theatre actor . He had a guest @-@ starring role on the television series The Bill in 2000 . This was followed by a starring role in the play Herons written by Simon Stephens , which was performed in 2001 at the Royal Court Theatre . He had a guest role in the television series Judge John Deed in 2002 . In 2004 Boulter landed a role as " Craig " in the episode " Teddy \'s Story " of the television series The Long Firm ; he starred alongside actors Mark Strong and Derek Jacobi . He was cast in the 2005 theatre productions of the Philip Ridley play Mercury Fur , which was performed at the Drum Theatre in Plymouth and the Menier Chocolate Factory in London . He was directed by John Tiffany and starred alongside Ben Whishaw , Shane Zaza , Harry Kent , Fraser Ayres , Sophie Stanton and Dominic Hall . \n',
 ' In 2006 , Boulter starred alongside Whishaw in the play Citizenship written by Mark Ravenhill 

We can identify the split between articles by looking out for the
article headings. In this dataset:  
- Headers have the format: `" = <HEADING> = \n"`  
- Subheadings have the format: `" = = <SUBHEADING> = = \n"`  
- Subsubheadings have the format: `" = = = <SUBSUBHEADING> = = = \n"`

In [29]:
articles = []

# Drop the 0th line as its empty
for line in ds["test"]["text"][1:]:
    # Identify headings, but exclude subheadings and subsubheadings
    if line[:3] == " = " and line[3] != "=":
        articles.append(line)
    else:
        articles[-1] += line

In [30]:
# Check the first 100 characters of the first 5 articles to make sure we split the dataset correctly
for article in articles[:5]:
    print(article[:100])

 = Robert Boulter = 
 Robert Boulter is an English film , television and theatre actor . He had a gu
 = Du Fu = 
 Du Fu ( Wade – Giles : Tu Fu ; Chinese : 杜甫 ; 712 – 770 ) was a prominent Chinese poet 
 = Kiss You ( One Direction song ) = 
 " Kiss You " is a song recorded by English @-@ Irish boy band
 = Ise @-@ class battleship = 
 The Ise @-@ class battleships ( 伊勢型戦艦 , Ise @-@ gata senkan ) were a
 = Dick Rifenburg = 
 Richard Gale " Dick " Rifenburg ( August 21 , 1926 – December 5 , 1994 ) was a

## Passing the articles through to Ollama

To use the ollama library we need to start an ollama server in the
background inside of our SLURM job. We can do this by running the
commands below in a **Jupyter Lab terminal**. This is not the same as
launching a terminal app as that will launch a new SLURM job instead.
Instead, select ‘Terminal’ from the Launcher after clicking on the ‘+’
button in the top left corner of your Jupyter Lab application.

We also need to point Ollama towards our shared model repository in
order to be able to import the models that we’ve downloaded for you.
Alternatively if you’d like to experiment with models on your own, feel
free to modify that path to a directory that you control. Note that LLMs
are big and they will count towards your disk quota.

`export OLLAMA_MODELS="/apps/ollama/models/"`  
`/apps/ollama/ollama serve`

In [31]:
import ollama
response = ollama.chat(model='llama3', messages=[
  {
    'role': 'user',
    'content': 'Why is the sky blue?',
  },
])
print(response)

{'model': 'llama3', 'created_at': '2024-08-14T05:34:36.062060907Z', 'message': {'role': 'assistant', 'content': "What a great question!\n\nThe sky appears blue because of a phenomenon called Rayleigh scattering. This is when short (blue) wavelengths of light are scattered more than longer (red) wavelengths by tiny molecules of gases in the atmosphere, such as nitrogen and oxygen.\n\nHere's what happens:\n\n1. When sunlight enters Earth's atmosphere, it encounters tiny molecules of gases like nitrogen (N2) and oxygen (O2).\n2. These molecules scatter the light in all directions.\n3. The shorter blue wavelengths are scattered more than longer red wavelengths because they have a higher energy and interact more strongly with the gas molecules.\n4. As a result, our eyes perceive the scattered blue light as a blue color, making the sky appear blue.\n\nThis scattering effect is most pronounced when the sun is overhead (at an angle of about 90 degrees), which is why the sky often appears more 

We can then formulate our request to the LLM using the articles from our
dataset and ask for it’s summary

In [32]:
response = ollama.chat(model='llama3', messages=[
  {
    'role': 'user',
    'content': "Please summarise this article:\n\n\n" + articles[0],
  },
])
print(response['message']['content'])

The article is about the career of Robert Boulter, an English actor. Here's a summary:

**Early Career (2000-2005)**

* Guest starred in TV series "The Bill" (2000)
* Starred in play "Herons" at Royal Court Theatre (2001)
* Appeared in TV series "Judge John Deed" (2002) and "The Bill" (2003)
* Had recurring role on "The Bill" (2003)

**Middle Career (2006-Present)**

* Starred in play "Citizenship" at National Theatre (2006)
* Appeared in TV series "Doctors" (2006)
* Starred in play "How to Curse" at Bush Theatre (2007)
* Starred in two films: "Daylight Robbery" and "Donkey Punch" (2008)
* Guest starred on TV series "Waking the Dead" (2008) and "Survivors" (2008)
* Had recurring role on TV series "Casualty" (2010)
* Starred in film "Mercenaries" (2011)

**Theatre Credits**

* "Mercury Fur" at Drum Theatre and Menier Chocolate Factory (2005)
* "How to Curse" at Bush Theatre (2007)

Overall, Robert Boulter has had a diverse career, appearing in TV shows, films, and theatre productions. H

In [ ]:
# And finally we can automate 
responses = []
for article in articles:
    response = ollama.chat(model='llama3', messages=[
        {
            'role': 'user',
            'content': "Please summarise this article:\n\n\n" + article,
        },
    ])
    responses.append(response['message']['content'])

In [36]:
# Print the first 5 summaries
for response in responses[:5]:
    print(response)
    print("======================")

The article is a biography of English actor Robert Boulter. Here's a summary:

**Early Career (2000-2005)**

* Guest star role on TV series "The Bill" in 2000
* Starred in play "Herons" by Simon Stephens at Royal Court Theatre in 2001
* Appeared in TV series "Judge John Deed" in 2002 as "Addem Armitage"
* Had recurring role on TV series "The Bill" in 2003 and 2004

**Mid-Career (2006-2010)**

* Starred in play "Citizenship" by Mark Ravenhill at National Theatre in 2006
* Appeared in TV series "Doctors" in 2006 as "Jason Tyler"
* Starred in play "How to Curse" directed by Josie Rourke at Bush Theatre in 2007
* Starred in two films: "Daylight Robbery" (2008) and "Donkey Punch" (2008)
* Guest-starred on TV series "Waking the Dead" in 2008 as "Jimmy Dearden"
* Appeared on TV series "Survivors" in 2008 as "Neil"

**Recent Career (2010-Present)**

* Had recurring role on TV series "Casualty" in 2010 as "Kieron Fletcher", playing an emergency physician
* Starred in film "Mercenaries" directed